In [1]:
import librosa

In [2]:
import pandas as pd
import numpy as np
import librosa

In [3]:
df = pd.read_csv('SEP-28k_labels.csv')
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [4]:
df.shape

(28177, 17)

In [5]:
df.columns

Index(['Show', 'EpId', 'ClipId', 'Start', 'Stop', 'Unsure', 'PoorAudioQuality',
       'Prolongation', 'Block', 'SoundRep', 'WordRep', 'DifficultToUnderstand',
       'Interjection', 'NoStutteredWords', 'NaturalPause', 'Music',
       'NoSpeech'],
      dtype='object')

## Adding Name Column

In [6]:
df['Name'] = df[df.columns[0:3]].apply(
    lambda x: '_'.join(x.dropna().astype(str)),
    axis=1
)
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech,Name
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0,HeStutters_0_0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0,HeStutters_0_1
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0,HeStutters_0_2
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0,HeStutters_0_3
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0,HeStutters_0_4


In [7]:
df['Name']

0                HeStutters_0_0
1                HeStutters_0_1
2                HeStutters_0_2
3                HeStutters_0_3
4                HeStutters_0_4
                  ...          
28172    WomenWhoStutter_109_35
28173    WomenWhoStutter_109_36
28174    WomenWhoStutter_109_37
28175    WomenWhoStutter_109_38
28176    WomenWhoStutter_109_39
Name: Name, Length: 28177, dtype: object

## Removing Empty Audios and there Dataset Entries

In [8]:
import os
os.stat("clip/HeStutters/0/HeStutters_0_9.wav").st_size

# empty files have st_size of 44

96044

In [9]:
os.stat("clip/HeStutters/1/HeStutters_1_1.wav").st_size

# non empty file

96044

## Put empty filenames in a list and ignore them while feature extraction and training

In [10]:
CLIPS_DIR = "clip/"
ignore_list = []
for filename in os.listdir(CLIPS_DIR):
    file_path = CLIPS_DIR + filename
    if 'FluencyBank' not in filename:
        if os.stat(file_path).st_size == 44:
            ignore_list.append(filename)
            filename = filename[:-4]
            df = df[df.Name != filename]

print(len(ignore_list))

0


In [11]:
df.shape

# 255 rows removed from df as well

(28177, 18)

## MFCC Feature Extraction

In [12]:
import librosa
import numpy as np
from tqdm.notebook import tqdm

features = {}
directory = CLIPS_DIR

for filename in tqdm(os.listdir(CLIPS_DIR)):
    for a in tqdm(os.listdir("clip/"+filename+'/')):
        for b in tqdm(os.listdir("clip/"+filename+'/'+a+'/')):
            if 'FluencyBank' not in filename and ignore_list.count(filename + '.wav') == 0:
                audio, sample_rate = librosa.load("clip/" + filename + '/' + a + '/' + b, res_type='kaiser_fast', sr=None)
                mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13).T,axis=0)
                features[b[:-4]] = mfccs

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/201 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

In [13]:
print(len(features))

21736


In [14]:
print(features)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
len(features['HeStutters_1_1'])

13

## Making Dataset from Features

In [16]:
df_features = pd.DataFrame.from_dict(features)
df_features = df_features.transpose()
df_features

,0,1,2,3,4,5,6,7,8,9,10,11,12
HeStutters_0_23,-314.641907,61.979465,-0.235741,12.721733,-0.621172,-5.817106,-13.321684,-4.004771,-12.716882,-8.100995,-4.890786,-7.404508,-7.515119
HeStutters_0_0,-392.172791,63.350689,4.833431,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
HeStutters_0_1,-336.544678,60.795284,-2.917545,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
HeStutters_0_10,-409.439911,61.835724,11.941999,23.786074,-0.019960,-0.494438,-7.668434,4.261880,-15.007185,-3.078003,-1.703809,-9.908122,-4.152675
HeStutters_0_11,-310.758362,27.546112,-17.378851,24.965218,-20.023012,1.167747,-13.532715,-7.309593,-3.097385,-5.110380,-7.832861,-5.489938,-3.351382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WomenWhoStutter_99_30,-349.252258,63.760883,-35.519657,24.252199,-17.412300,-2.188475,-0.694868,-1.097295,13.670923,-3.851459,-3.219055,-6.764341,1.756579
WomenWhoStutter_99_31,-310.781036,18.843845,-8.842707,33.856659,-17.807787,2.555339,-0.644569,-7.980123,-3.171874,-0.392410,-8.492411,-3.692825,-4.162845
WomenWhoStutter_99_32,-380.588593,16.248068,-16.056593,35.585838,-1.616079,6.364790,3.964227,0.626711,5.347195,-4.598023,-5.764363,-2.281778,-2.297391
WomenWhoStutter_99_33,-299.443451,4.185234,-45.925980,27.375067,-17.340590,6.882776,3.560043,-1.974369,18.104261,-8.154037,-2.990829,-6.299887,1.728673


In [17]:
df_features = df_features.reset_index()

In [18]:
df_features

,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,HeStutters_0_23,-314.641907,61.979465,-0.235741,12.721733,-0.621172,-5.817106,-13.321684,-4.004771,-12.716882,-8.100995,-4.890786,-7.404508,-7.515119
1,HeStutters_0_0,-392.172791,63.350689,4.833431,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
2,HeStutters_0_1,-336.544678,60.795284,-2.917545,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
3,HeStutters_0_10,-409.439911,61.835724,11.941999,23.786074,-0.019960,-0.494438,-7.668434,4.261880,-15.007185,-3.078003,-1.703809,-9.908122,-4.152675
4,HeStutters_0_11,-310.758362,27.546112,-17.378851,24.965218,-20.023012,1.167747,-13.532715,-7.309593,-3.097385,-5.110380,-7.832861,-5.489938,-3.351382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21731,WomenWhoStutter_99_30,-349.252258,63.760883,-35.519657,24.252199,-17.412300,-2.188475,-0.694868,-1.097295,13.670923,-3.851459,-3.219055,-6.764341,1.756579
21732,WomenWhoStutter_99_31,-310.781036,18.843845,-8.842707,33.856659,-17.807787,2.555339,-0.644569,-7.980123,-3.171874,-0.392410,-8.492411,-3.692825,-4.162845
21733,WomenWhoStutter_99_32,-380.588593,16.248068,-16.056593,35.585838,-1.616079,6.364790,3.964227,0.626711,5.347195,-4.598023,-5.764363,-2.281778,-2.297391
21734,WomenWhoStutter_99_33,-299.443451,4.185234,-45.925980,27.375067,-17.340590,6.882776,3.560043,-1.974369,18.104261,-8.154037,-2.990829,-6.299887,1.728673


In [19]:
df_features = df_features.sort_values(by='index')
df_features

,index,0,1,2,3,4,5,6,7,8,9,10,11,12
3684,HVSA_0_0,-672.897095,42.572784,-9.060439,17.092854,5.601506,11.969574,5.087627,1.287422,-1.221630,5.906602,0.396098,2.618646,2.006739
3685,HVSA_0_1,-463.573669,69.182426,-18.914116,22.859167,0.735868,-7.253482,-7.202471,-5.319007,-5.854908,4.434800,-2.941299,0.067582,-10.047692
3686,HVSA_0_10,-452.379944,58.461617,-26.529619,10.806691,1.040473,-11.684329,-15.975926,-17.110899,-8.533467,3.804462,-0.904730,1.546427,-5.672745
3687,HVSA_0_100,-454.134918,85.565826,-16.623894,23.212719,-7.808586,-7.204661,-8.445378,-17.406126,-16.752789,-5.885397,0.342330,1.163908,-5.125210
3688,HVSA_0_101,-419.757965,73.650093,-20.263416,30.691795,-5.919299,-15.747583,-17.799961,-13.380462,-10.132165,-2.647573,1.943859,-0.861621,-5.781013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20863,WomenWhoStutter_9_5,-331.060883,8.423852,-32.912979,25.618399,-23.380960,2.367096,-14.927189,-5.307941,0.028746,-7.770761,-5.659317,-3.837930,-1.988690
20864,WomenWhoStutter_9_6,-287.171539,24.612059,11.593957,-2.373049,-19.041893,1.381284,-8.410903,-6.094375,-21.232729,1.682852,-18.440504,5.257711,-14.749590
20865,WomenWhoStutter_9_7,-284.419769,69.935936,4.447401,25.512424,-23.518784,-3.872571,2.403356,-14.279772,-22.612999,5.954080,-23.500135,-1.513236,-22.199059
20866,WomenWhoStutter_9_8,-359.677887,48.367863,14.129713,3.370583,-19.779791,1.081605,-1.616947,-4.654747,-21.594097,-1.051946,-16.132671,8.522419,-16.694715


In [20]:
df_features['index']

3684                HVSA_0_0
3685                HVSA_0_1
3686               HVSA_0_10
3687              HVSA_0_100
3688              HVSA_0_101
                ...         
20863    WomenWhoStutter_9_5
20864    WomenWhoStutter_9_6
20865    WomenWhoStutter_9_7
20866    WomenWhoStutter_9_8
20867    WomenWhoStutter_9_9
Name: index, Length: 21736, dtype: object

In [21]:
df['Name']

0                HeStutters_0_0
1                HeStutters_0_1
2                HeStutters_0_2
3                HeStutters_0_3
4                HeStutters_0_4
                  ...          
28172    WomenWhoStutter_109_35
28173    WomenWhoStutter_109_36
28174    WomenWhoStutter_109_37
28175    WomenWhoStutter_109_38
28176    WomenWhoStutter_109_39
Name: Name, Length: 28177, dtype: object

## Applying Inner Join on the dataframes

In [22]:
df_features.rename(columns = {'index':'Name'}, inplace = True)
df_features

,Name,0,1,2,3,4,5,6,7,8,9,10,11,12
3684,HVSA_0_0,-672.897095,42.572784,-9.060439,17.092854,5.601506,11.969574,5.087627,1.287422,-1.221630,5.906602,0.396098,2.618646,2.006739
3685,HVSA_0_1,-463.573669,69.182426,-18.914116,22.859167,0.735868,-7.253482,-7.202471,-5.319007,-5.854908,4.434800,-2.941299,0.067582,-10.047692
3686,HVSA_0_10,-452.379944,58.461617,-26.529619,10.806691,1.040473,-11.684329,-15.975926,-17.110899,-8.533467,3.804462,-0.904730,1.546427,-5.672745
3687,HVSA_0_100,-454.134918,85.565826,-16.623894,23.212719,-7.808586,-7.204661,-8.445378,-17.406126,-16.752789,-5.885397,0.342330,1.163908,-5.125210
3688,HVSA_0_101,-419.757965,73.650093,-20.263416,30.691795,-5.919299,-15.747583,-17.799961,-13.380462,-10.132165,-2.647573,1.943859,-0.861621,-5.781013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20863,WomenWhoStutter_9_5,-331.060883,8.423852,-32.912979,25.618399,-23.380960,2.367096,-14.927189,-5.307941,0.028746,-7.770761,-5.659317,-3.837930,-1.988690
20864,WomenWhoStutter_9_6,-287.171539,24.612059,11.593957,-2.373049,-19.041893,1.381284,-8.410903,-6.094375,-21.232729,1.682852,-18.440504,5.257711,-14.749590
20865,WomenWhoStutter_9_7,-284.419769,69.935936,4.447401,25.512424,-23.518784,-3.872571,2.403356,-14.279772,-22.612999,5.954080,-23.500135,-1.513236,-22.199059
20866,WomenWhoStutter_9_8,-359.677887,48.367863,14.129713,3.370583,-19.779791,1.081605,-1.616947,-4.654747,-21.594097,-1.051946,-16.132671,8.522419,-16.694715


In [23]:
df_final = pd.merge(df, df_features, how='inner', on='Name')
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21731,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143101,-26.160055,3.709430,-15.719262,2.436124,-1.615821,-5.105482,-10.272832,-3.074108,-6.375843
21732,WomenWhoStutter,109,36,26168480,26216480,0,0,1,1,3,...,29.588446,-10.896355,-14.222120,-9.167952,-1.676773,-8.374957,-6.297094,1.265485,-4.014944,-0.196149
21733,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942730,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21734,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132101,-4.451934,-1.437802,-11.729773,-6.692393,0.137587,-3.664662


In [24]:
df_final.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803761,-6.036355


In [25]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21736 entries, 0 to 21735
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Show                   21736 non-null  object 
 1   EpId                   21736 non-null  int64  
 2   ClipId                 21736 non-null  int64  
 3   Start                  21736 non-null  int64  
 4   Stop                   21736 non-null  int64  
 5   Unsure                 21736 non-null  int64  
 6   PoorAudioQuality       21736 non-null  int64  
 7   Prolongation           21736 non-null  int64  
 8   Block                  21736 non-null  int64  
 9   SoundRep               21736 non-null  int64  
 10  WordRep                21736 non-null  int64  
 11  DifficultToUnderstand  21736 non-null  int64  
 12  Interjection           21736 non-null  int64  
 13  NoStutteredWords       21736 non-null  int64  
 14  NaturalPause           21736 non-null  int64  
 15  Mu

In [26]:
# df_final['Stutter'] = df['WordRep'] + df['SoundRep'] + df['Prolongation'] + df['Interjection']
# df_final

In [27]:
# df_final['Stutter'].value_counts()

In [28]:
# df_final = df_final[df_final.Stutter != 0]
# Remove Non-Stuttered Clips

# df_final.loc[df_final['Stutter'] >= 1.0, 'Stutter'] = 1.0
# df_final['Stutter'].value_counts()

In [29]:
df_final = df_final[df_final.PoorAudioQuality == 0]
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21731,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143101,-26.160055,3.709430,-15.719262,2.436124,-1.615821,-5.105482,-10.272832,-3.074108,-6.375843
21732,WomenWhoStutter,109,36,26168480,26216480,0,0,1,1,3,...,29.588446,-10.896355,-14.222120,-9.167952,-1.676773,-8.374957,-6.297094,1.265485,-4.014944,-0.196149
21733,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942730,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21734,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132101,-4.451934,-1.437802,-11.729773,-6.692393,0.137587,-3.664662


In [30]:
df_final = df_final[df_final.DifficultToUnderstand == 0]
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21730,WomenWhoStutter,109,34,24781440,24829440,0,0,0,0,0,...,5.831639,-24.096672,-9.137759,-8.294925,-17.235229,-3.540464,-5.074893,-3.828139,-5.524148,-5.444070
21731,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143101,-26.160055,3.709430,-15.719262,2.436124,-1.615821,-5.105482,-10.272832,-3.074108,-6.375843
21733,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942730,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21734,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132101,-4.451934,-1.437802,-11.729773,-6.692393,0.137587,-3.664662


In [31]:
df_final = df_final[df_final.Music == 0]
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21730,WomenWhoStutter,109,34,24781440,24829440,0,0,0,0,0,...,5.831639,-24.096672,-9.137759,-8.294925,-17.235229,-3.540464,-5.074893,-3.828139,-5.524148,-5.444070
21731,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143101,-26.160055,3.709430,-15.719262,2.436124,-1.615821,-5.105482,-10.272832,-3.074108,-6.375843
21733,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942730,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21734,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132101,-4.451934,-1.437802,-11.729773,-6.692393,0.137587,-3.664662


In [32]:
df_final = df_final[df_final.NoSpeech == 0]
df_final

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037708,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803761,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21730,WomenWhoStutter,109,34,24781440,24829440,0,0,0,0,0,...,5.831639,-24.096672,-9.137759,-8.294925,-17.235229,-3.540464,-5.074893,-3.828139,-5.524148,-5.444070
21731,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143101,-26.160055,3.709430,-15.719262,2.436124,-1.615821,-5.105482,-10.272832,-3.074108,-6.375843
21733,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942730,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
21734,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132101,-4.451934,-1.437802,-11.729773,-6.692393,0.137587,-3.664662


In [33]:
df_final.to_csv("sep28k-mfcc.csv",index=False)

## Model Creation & Training for SoundRep Classification

In [34]:
import pandas as pd
df = pd.read_csv("sep28k-mfcc.csv")
df

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,...,6.977844,2.169690,-3.574423,-7.091357,-1.978344,-8.984580,-4.724132,-3.345276,-6.573854,-7.026947
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,...,10.253622,-11.466489,2.707656,-10.675123,-4.653667,-14.433005,-5.926201,-1.934813,-8.946336,-8.399075
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037710,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803762,-6.036355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16318,WomenWhoStutter,109,34,24781440,24829440,0,0,0,0,0,...,5.831639,-24.096672,-9.137759,-8.294925,-17.235230,-3.540464,-5.074892,-3.828139,-5.524148,-5.444070
16319,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,...,20.143100,-26.160055,3.709430,-15.719262,2.436124,-1.615821,-5.105482,-10.272832,-3.074108,-6.375843
16320,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,...,9.234286,-27.815256,-5.439774,-17.093494,-8.942730,-1.638088,-3.736744,-0.968271,-7.699818,0.135875
16321,WomenWhoStutter,109,38,28479840,28527840,0,0,1,0,0,...,26.443172,-24.452421,-10.057486,-10.132101,-4.451934,-1.437802,-11.729773,-6.692393,0.137587,-3.664662


In [35]:
df = df[df['NoStutteredWords'] != 0]
df = df[df['NaturalPause'] == 0]
df = df[df['Interjection'] == 0]
df = df[df['Prolongation'] == 0]
df = df[df['WordRep'] == 0]
df = df[df['Block'] == 0]
df

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037710,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803762,-6.036355
6,HeStutters,0,6,37251200,37299200,0,0,0,0,0,...,4.350783,-14.230150,-3.446104,-3.806112,-6.410494,-7.694991,-1.443513,-7.135467,-3.231778,-8.599639
9,HeStutters,0,9,41417440,41465440,0,0,0,0,0,...,10.802254,0.253165,-0.571133,-7.632134,-2.497033,-8.864999,-2.741921,-5.094120,-9.865229,-3.816170
11,HeStutters,0,11,42928800,42976800,0,0,0,0,0,...,24.965218,-20.023012,1.167747,-13.532715,-7.309593,-3.097385,-5.110380,-7.832861,-5.489938,-3.351382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,WomenWhoStutter,109,24,16179040,16227040,0,0,0,0,0,...,26.002827,-16.124668,-14.775861,-19.643862,-7.232726,-7.771799,-8.517433,1.568335,-6.449186,1.097754
16309,WomenWhoStutter,109,25,16224160,16272160,0,0,0,0,0,...,8.484849,-29.523224,-2.754617,-15.226082,-7.238287,1.549171,-8.778543,-10.499634,-2.756247,-6.887878
16315,WomenWhoStutter,109,31,20476320,20524320,0,0,0,0,0,...,16.823680,-23.059666,-6.954729,-10.113519,-13.114539,0.588434,-15.267725,-10.102099,-0.440946,0.197361
16317,WomenWhoStutter,109,33,24747360,24795360,0,0,0,0,0,...,18.849432,-23.086039,2.158883,-8.471071,1.332330,2.440598,-1.968042,-7.462180,0.142055,-4.275077


In [36]:
df.loc[df['SoundRep'] >= 1.0, 'SoundRep'] = 1.0
df['SoundRep'].value_counts()

0    1684
1     121
Name: SoundRep, dtype: int64

In [37]:
X = df.iloc[: , -13:]
X

,0,1,2,3,4,5,6,7,8,9,10,11,12
2,-313.66693,62.418663,-1.334002,36.037710,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
4,-341.35330,68.858000,-3.959720,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803762,-6.036355
6,-351.60030,79.161500,10.344581,4.350783,-14.230150,-3.446104,-3.806112,-6.410494,-7.694991,-1.443513,-7.135467,-3.231778,-8.599639
9,-322.53464,84.011630,-0.464124,10.802254,0.253165,-0.571133,-7.632134,-2.497033,-8.864999,-2.741921,-5.094120,-9.865229,-3.816170
11,-310.75836,27.546112,-17.378850,24.965218,-20.023012,1.167747,-13.532715,-7.309593,-3.097385,-5.110380,-7.832861,-5.489938,-3.351382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,-396.81607,47.699245,-28.230492,26.002827,-16.124668,-14.775861,-19.643862,-7.232726,-7.771799,-8.517433,1.568335,-6.449186,1.097754
16309,-366.09630,59.089397,-23.645912,8.484849,-29.523224,-2.754617,-15.226082,-7.238287,1.549171,-8.778543,-10.499634,-2.756247,-6.887878
16315,-316.00030,42.245895,-40.056310,16.823680,-23.059666,-6.954729,-10.113519,-13.114539,0.588434,-15.267725,-10.102099,-0.440946,0.197361
16317,-350.42902,27.453814,-35.155933,18.849432,-23.086039,2.158883,-8.471071,1.332330,2.440598,-1.968042,-7.462180,0.142055,-4.275077


In [38]:
y = df['SoundRep']
y

2        0
4        0
6        0
9        0
11       0
        ..
16308    0
16309    0
16315    0
16317    0
16322    0
Name: SoundRep, Length: 1805, dtype: int64

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.4, random_state=42)

In [40]:
from collections import Counter
Counter(y_train)

Counter({0: 1017, 1: 66})

In [41]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy',random_state=5)
clf.fit(X_train, y_train)
clf.score(X_test,y_test) * 100

86.98060941828255

In [42]:
y_pred = np.array(clf.predict(X_test))
y_actual = np.array(y_test)
print(Counter(y_pred))
print(Counter(y_actual))

Counter({0: 675, 1: 47})
Counter({0: 667, 1: 55})


## Model Creation & Training for WordRep Classification

In [43]:
import pandas as pd
df = pd.read_csv('sep28k-mfcc.csv')

In [44]:
df.columns

Index(['Show', 'EpId', 'ClipId', 'Start', 'Stop', 'Unsure', 'PoorAudioQuality',
       'Prolongation', 'Block', 'SoundRep', 'WordRep', 'DifficultToUnderstand',
       'Interjection', 'NoStutteredWords', 'NaturalPause', 'Music', 'NoSpeech',
       'Name', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12'],
      dtype='object')

In [45]:
df.shape

(16323, 31)

In [46]:
df = df[df['NoStutteredWords'] != 0]
df = df[df['NaturalPause'] == 0]
df = df[df['Interjection'] == 0]
df = df[df['Prolongation'] == 0]
df = df[df['SoundRep'] == 0]
df = df[df['Block'] == 0]
df

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037710,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803762,-6.036355
6,HeStutters,0,6,37251200,37299200,0,0,0,0,0,...,4.350783,-14.230150,-3.446104,-3.806112,-6.410494,-7.694991,-1.443513,-7.135467,-3.231778,-8.599639
9,HeStutters,0,9,41417440,41465440,0,0,0,0,0,...,10.802254,0.253165,-0.571133,-7.632134,-2.497033,-8.864999,-2.741921,-5.094120,-9.865229,-3.816170
11,HeStutters,0,11,42928800,42976800,0,0,0,0,0,...,24.965218,-20.023012,1.167747,-13.532715,-7.309593,-3.097385,-5.110380,-7.832861,-5.489938,-3.351382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,WomenWhoStutter,109,24,16179040,16227040,0,0,0,0,0,...,26.002827,-16.124668,-14.775861,-19.643862,-7.232726,-7.771799,-8.517433,1.568335,-6.449186,1.097754
16309,WomenWhoStutter,109,25,16224160,16272160,0,0,0,0,0,...,8.484849,-29.523224,-2.754617,-15.226082,-7.238287,1.549171,-8.778543,-10.499634,-2.756247,-6.887878
16315,WomenWhoStutter,109,31,20476320,20524320,0,0,0,0,0,...,16.823680,-23.059666,-6.954729,-10.113519,-13.114539,0.588434,-15.267725,-10.102099,-0.440946,0.197361
16317,WomenWhoStutter,109,33,24747360,24795360,0,0,0,0,0,...,18.849432,-23.086039,2.158883,-8.471071,1.332330,2.440598,-1.968042,-7.462180,0.142055,-4.275077


In [47]:
df.loc[df['WordRep'] >= 1.0, 'WordRep'] = 1.0
df['WordRep'].value_counts()

0    1684
1     147
Name: WordRep, dtype: int64

In [48]:
X = df.iloc[: , -13:]
X

,0,1,2,3,4,5,6,7,8,9,10,11,12
2,-313.66693,62.418663,-1.334002,36.037710,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
4,-341.35330,68.858000,-3.959720,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803762,-6.036355
6,-351.60030,79.161500,10.344581,4.350783,-14.230150,-3.446104,-3.806112,-6.410494,-7.694991,-1.443513,-7.135467,-3.231778,-8.599639
9,-322.53464,84.011630,-0.464124,10.802254,0.253165,-0.571133,-7.632134,-2.497033,-8.864999,-2.741921,-5.094120,-9.865229,-3.816170
11,-310.75836,27.546112,-17.378850,24.965218,-20.023012,1.167747,-13.532715,-7.309593,-3.097385,-5.110380,-7.832861,-5.489938,-3.351382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,-396.81607,47.699245,-28.230492,26.002827,-16.124668,-14.775861,-19.643862,-7.232726,-7.771799,-8.517433,1.568335,-6.449186,1.097754
16309,-366.09630,59.089397,-23.645912,8.484849,-29.523224,-2.754617,-15.226082,-7.238287,1.549171,-8.778543,-10.499634,-2.756247,-6.887878
16315,-316.00030,42.245895,-40.056310,16.823680,-23.059666,-6.954729,-10.113519,-13.114539,0.588434,-15.267725,-10.102099,-0.440946,0.197361
16317,-350.42902,27.453814,-35.155933,18.849432,-23.086039,2.158883,-8.471071,1.332330,2.440598,-1.968042,-7.462180,0.142055,-4.275077


In [49]:
y = df['WordRep']
y

2        0
4        0
6        0
9        0
11       0
        ..
16308    0
16309    0
16315    0
16317    0
16322    0
Name: WordRep, Length: 1831, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.4, random_state=42)

from collections import Counter
Counter(y_train)

Counter({0: 1010, 1: 88})

In [51]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini',random_state=5)
# clf.fit(X_over, y_over)
# clf.fit(X_smote, y_smote)
# clf.fit(X_pipe, y_pipe)
clf.fit(X_train, y_train)
clf.score(X_test,y_test) * 100

84.31105047748977

In [52]:
y_pred = np.array(clf.predict(X_test))
y_actual = np.array(y_test)
print(Counter(y_pred))
print(Counter(y_actual))

Counter({0: 667, 1: 66})
Counter({0: 674, 1: 59})


## Model Creation & Training for Prolongation Classification

In [53]:
import pandas as pd
df = pd.read_csv('sep28k-mfcc.csv')

In [54]:
df.columns

Index(['Show', 'EpId', 'ClipId', 'Start', 'Stop', 'Unsure', 'PoorAudioQuality',
       'Prolongation', 'Block', 'SoundRep', 'WordRep', 'DifficultToUnderstand',
       'Interjection', 'NoStutteredWords', 'NaturalPause', 'Music', 'NoSpeech',
       'Name', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12'],
      dtype='object')

In [55]:
df.shape

(16323, 31)

In [56]:
df = df[df['NoStutteredWords'] != 0]
df = df[df['NaturalPause'] == 0]
df = df[df['Interjection'] == 0]
df = df[df['WordRep'] == 0]
df = df[df['SoundRep'] == 0]
df = df[df['Block'] == 0]
df

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,3,4,5,6,7,8,9,10,11,12
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,36.037710,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803762,-6.036355
5,HeStutters,0,5,36435040,36483040,0,0,1,0,0,...,22.589972,7.785811,-5.386620,-10.450265,-1.401688,-9.730509,-5.115982,-6.537759,-6.979672,-7.198001
6,HeStutters,0,6,37251200,37299200,0,0,0,0,0,...,4.350783,-14.230150,-3.446104,-3.806112,-6.410494,-7.694991,-1.443513,-7.135467,-3.231778,-8.599639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,WomenWhoStutter,109,24,16179040,16227040,0,0,0,0,0,...,26.002827,-16.124668,-14.775861,-19.643862,-7.232726,-7.771799,-8.517433,1.568335,-6.449186,1.097754
16309,WomenWhoStutter,109,25,16224160,16272160,0,0,0,0,0,...,8.484849,-29.523224,-2.754617,-15.226082,-7.238287,1.549171,-8.778543,-10.499634,-2.756247,-6.887878
16315,WomenWhoStutter,109,31,20476320,20524320,0,0,0,0,0,...,16.823680,-23.059666,-6.954729,-10.113519,-13.114539,0.588434,-15.267725,-10.102099,-0.440946,0.197361
16317,WomenWhoStutter,109,33,24747360,24795360,0,0,0,0,0,...,18.849432,-23.086039,2.158883,-8.471071,1.332330,2.440598,-1.968042,-7.462180,0.142055,-4.275077


In [57]:
df.loc[df['Prolongation'] >= 1.0, 'Prolongation'] = 1.0
df['Prolongation'].value_counts()

0    1684
1     690
Name: Prolongation, dtype: int64

In [58]:
X = df.iloc[: , -13:]
X

,0,1,2,3,4,5,6,7,8,9,10,11,12
2,-313.66693,62.418663,-1.334002,36.037710,-2.650917,-11.021187,-9.310137,-1.562138,-13.035163,-8.655548,-4.336683,-7.372598,-7.138989
3,-352.49725,50.207840,-5.879061,3.193294,-4.616055,-5.762987,-4.187028,-0.715044,-14.383714,-3.304795,-4.146274,-8.412235,-2.445787
4,-341.35330,68.858000,-3.959720,8.430685,-0.561030,-4.473869,-9.220818,-3.201562,-11.855061,-4.699773,-2.414847,-7.803762,-6.036355
5,-350.96436,70.840900,1.759576,22.589972,7.785811,-5.386620,-10.450265,-1.401688,-9.730509,-5.115982,-6.537759,-6.979672,-7.198001
6,-351.60030,79.161500,10.344581,4.350783,-14.230150,-3.446104,-3.806112,-6.410494,-7.694991,-1.443513,-7.135467,-3.231778,-8.599639
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,-396.81607,47.699245,-28.230492,26.002827,-16.124668,-14.775861,-19.643862,-7.232726,-7.771799,-8.517433,1.568335,-6.449186,1.097754
16309,-366.09630,59.089397,-23.645912,8.484849,-29.523224,-2.754617,-15.226082,-7.238287,1.549171,-8.778543,-10.499634,-2.756247,-6.887878
16315,-316.00030,42.245895,-40.056310,16.823680,-23.059666,-6.954729,-10.113519,-13.114539,0.588434,-15.267725,-10.102099,-0.440946,0.197361
16317,-350.42902,27.453814,-35.155933,18.849432,-23.086039,2.158883,-8.471071,1.332330,2.440598,-1.968042,-7.462180,0.142055,-4.275077


In [59]:
y = df['Prolongation']
y

2        0
3        1
4        0
5        1
6        0
        ..
16308    0
16309    0
16315    0
16317    0
16322    0
Name: Prolongation, Length: 2374, dtype: int64

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.4, random_state=42)

from collections import Counter
Counter(y_train)

Counter({1: 398, 0: 1026})

In [61]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
knn.score(X_test, y_test) * 100

/home/kakeshi/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


63.68421052631579

In [62]:
import numpy as np
y_pred = np.array(knn.predict(X_test))
y_actual = np.array(y_test)
print(Counter(y_pred))
print(Counter(y_actual))

Counter({0: 785, 1: 165})
Counter({0: 658, 1: 292})


/home/kakeshi/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
